In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Dropout
from tensorflow.python.keras.layers import Activation
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [3]:
monthly_sales = pd.read_csv('https://raw.githubusercontent.com/yuetongzhang/Costco_Yuetong-Zhang/main/Costco_Yuetong%20Zhang/Costco_Monthly%20Sales%20from%202012%20to%202021.csv')
monthly_sales1=monthly_sales.drop(columns=['Growth Rate','Date'])
monthly_sales1=monthly_sales1.dropna()
monthly_sales_df=pd.DataFrame(monthly_sales1)
monthly_sales_df = monthly_sales_df.rename(columns={'Net Sales (billions)': 'Total Sales'})
monthly_sales_df = monthly_sales_df[['Total Sales', 'Year', 'Month']]
#print(monthly_sales_df)
monthly_sales_df.head()

,Total Sales,Year,Month
0,7.00,2012,1
2,9.13,2012,3
3,7.25,2012,4
4,7.67,2012,5
5,9.18,2012,6


In [4]:
real_rate=pd.read_csv('https://raw.githubusercontent.com/RamenMode/NeuralNetsSales/main/100-EmploymentRateReal.csv')
df_real_rate=pd.DataFrame(real_rate)
df_real_rate.head()
irate= df_real_rate.iloc[:,3]
monthly_sales_df['Inflation Rate']=irate
print(monthly_sales_df)

     Total Sales  Year  Month  Inflation Rate
0           7.00  2012      1             2.9
2           9.13  2012      3             2.3
3           7.25  2012      4             1.7
4           7.67  2012      5             1.7
5           9.18  2012      6             1.4
..           ...   ...    ...             ...
109        14.05  2021      2             2.6
110        18.21  2021      3             4.2
111        15.21  2021      4             5.0
112        15.59  2021      5             5.4
113        18.92  2021      6             NaN

[113 rows x 4 columns]


In [5]:
len(monthly_sales_df)

113

In [6]:
data_2020 = monthly_sales_df.loc[monthly_sales_df['Year'] == 2020]
Y_2020 = data_2020['Total Sales']
X_2020 = data_2020[['Month','Year','Inflation Rate']]

In [7]:
data_before = monthly_sales_df.loc[monthly_sales_df['Year'] < 2020]
Y_before = data_before['Total Sales']
X_before = data_before[['Month','Year','Inflation Rate']]

In [8]:
XTrain, XTest, YTrain, YTest = train_test_split(X_before, Y_before, test_size = 0.2, shuffle = True)
#XTrain = X_before
#YTrain = Y_before

model0 = Sequential()
model0.add(Dense(450, activation = 'relu', input_dim=3))
model0.add(Dense(200, activation = 'relu'))
model0.add(Dropout(0.1))
model0.add(Dense(100, activation = 'relu'))
model0.add(Dropout(0.2))
model0.add(Dense(1, activation = 'linear'))
model0.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
loss30=0
loss2020=0
for i in range(30):
  model0.fit(XTrain, YTrain, epochs=100, batch_size=128)
  loss1 = model0.evaluate(XTest, YTest)
  loss2 = model0.evaluate(X_2020, Y_2020)
  loss30+=loss1
  loss2020+=loss2
  
  
  

In [10]:
print(np.sqrt(loss30/30))

2.5918146197495653


In [13]:
#loss = model0.evaluate(XTest, YTest)
print(np.sqrt(loss30/30))
print(monthly_sales_df['Total Sales'].std())
predict_2020 = model0.predict(X_2020)
r2_2020 = r2_score(Y_2020, predict_2020)

2.5918146197495653
2.7057365488546443


In [15]:
XTest = X_2020
YTest = Y_2020
#loss2020 = model0.evaluate(XTest, YTest)
print(np.sqrt(loss2020/30))
print(monthly_sales_df['Total Sales'].std())

5.670230685278303
2.7057365488546443


In [ ]:
data_2021 = monthly_sales_df.loc[monthly_sales_df['Year'] == 2021]
Y_2021 = data_2021['Net Sales (billions)']
X_2021 = data_2021[['Month','Year']]